In [63]:
import pandas as pd;
import numpy as np
import scipy.stats as st
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools    
import seaborn as sns
import matplotlib.pyplot as plt


import pandas as pd
import plotly.express as px
import json
import seaborn as sns

import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import plotly.graph_objects as go


In [64]:
def connection(query):
    con = psycopg2.connect(host='35.247.202.234', database='dw',
    user='consulta', password='teste123')
    cur = con.cursor()
    sql = query
    cur.execute(sql)
    data = cur.fetchall()
    cols = []
    for elt in cur.description:
        cols.append(elt[0])

    df = pd.DataFrame(data=data,columns=cols)
    con.close()
    return df


In [ ]:
##############Descritivo Digitalizacoes####################
query = 'SELECT * FROM digitalizacoes_firebase.avaliacao;'
data_digitalizacoes = connection(query)


In [4]:
# Check duplicidade
df_nan = data_digitalizacoes[data_digitalizacoes['arquivos_separados'].isna()]
df = data_digitalizacoes[~data_digitalizacoes['arquivos_separados'].isna()]
#print("Check duplicidade: ", df[['arquivos_separados']].duplicated().any())
df_drop_duplicates = df.drop_duplicates(subset=['arquivos_separados'])


v1 = df_drop_duplicates['arquivos_separados'].count()
v2 = df_nan['arquivos_separados'].isna().sum()


Valores Digitalizados 473156 e Nao digitalizados 1


In [41]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(['Redações Digitalizadas','Redações Não Digitalizadas']),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[v1, v2],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(title_text="Descritivo das digitalizações das Redações",title_x=0.5) 

fig.show()

In [6]:
##################Descritivo APA##################
query = 'SELECT * FROM devolutivas_apa.municipio;'
municipios =  connection(query);
query = 'SELECT * FROM devolutivas_apa.dados_ia_apa;'
data_apa = connection(query);

In [7]:
dict_uf= {11:'RO',12:'AC',13:'AM',14:'RR',15:'PA',16:'AP',17:'TO',21:'MA',22:'PI',23:'CE',24:'RN',
              25:'PB',26:'PE',27:'AL',28:'SE',29:'BA',31:'MG',32:'ES',33:'RJ',35:'SP',41:'PR',42:'SC',43:'RS',50:'MS',
              51:'MT',52:'GO',53:'DF',}
data_apa = data_apa.replace({"cod_estado": dict_uf})


In [8]:
dict_municipio = {}
for code in set(data_apa['codigo_cidade']):
    for codigo,nome in zip(municipios.id_municipio,municipios.nome):
        if(code==codigo):
            dict_municipio[code] = nome;
data_apa=data_apa.replace({'codigo_cidade': dict_municipio})

#Remover Duplicatas
data_apa_duplicate = data_apa.drop_duplicates(subset=['nome','level_pontuacao','created_at'])
result = data_apa_duplicate.groupby(['nome','level_pontuacao','created_at'],group_keys=True).apply(lambda x: x)

In [61]:
result = data_apa_duplicate.groupby(['cod_estado',]).agg({'nome':["nunique"],"turma": ["nunique"],"codigo_cidade": ["nunique"],'escola': ["nunique"]},split_out=4)
result.reset_index(inplace=True)
result.columns = ['Estado','Quantidade de Alunos','Quantidade de Turmas','Quantidade de Cidades','Quantidade de Escolas']
fig = go.Figure(data=[go.Table(
    header=dict(values=list(result.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[result.Estado, result['Quantidade de Alunos'],result['Quantidade de Turmas'], result['Quantidade de Cidades'],result['Quantidade de Escolas']],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(title_text="Demostrativo da quantidade de Alunos, Turmas, Cidades e Escolas por Estado",title_x=0.5) 

fig.show()

In [60]:
df = data_apa_duplicate.groupby(['cod_estado']).agg({'codigo_cidade': lambda x: x.nunique(),
                                                     "escola": lambda x: x.nunique(),
                                                      'nome': lambda x: x.nunique()})

df = df.reset_index()
df.columns = ['cod_estado','N° de Cidades','N° de Escolas','N° de Estudantes']
#df.sort_values(['Estado','Qt'], ascending=[True, False])

cross_tab_prop = pd.crosstab(index=data_apa_duplicate['cod_estado'],
                             columns=data_apa_duplicate['level_pontuacao'],
                             normalize="index")

cross_tab = pd.crosstab(index=data_apa_duplicate['cod_estado'],
                             columns=data_apa_duplicate['level_pontuacao'],)

#cross_tab_prop
cross_tab_prop.reset_index(inplace=True)

df_aux = pd.merge(df,cross_tab_prop[['cod_estado','A','B','D']],on='cod_estado', how='left')
df_aux.rename(columns={'cod_estado': 'Estado','A':'Pontuação A(%)',
                       'B':'Pontuação B(%)','D':'Pontuação D(%)'}, inplace=True)

valorA = cross_tab[['A']].agg('sum', axis=0)[0]
valorB = cross_tab[['B']].agg('sum', axis=0)[0]
valorD = cross_tab[['D']].agg('sum', axis=0)[0]


total = valorA+valorB+valorD
df_new_row = pd.DataFrame.from_records({'Estado':['Brasil'], 
                        'N° de Cidades':[df[['N° de Cidades']].agg('sum', axis=0)[0]],
                        'N° de Escolas':[df[['N° de Escolas']].agg('sum', axis=0)[0]],
                        'N° de Estudantes':[df[['N° de Estudantes']].agg('sum', axis=0)[0]],
                         'Pontuação A(%)':[valorA/total],
                         'Pontuação B(%)':[valorB/total],
                         'Pontuação D(%)':[valorD/total], 
                       })

df_aux = pd.concat([df_aux,df_new_row])

df_aux = df_aux.round(2)

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df_aux.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df_aux['Estado'],df_aux['N° de Cidades'], df_aux['N° de Escolas'], df_aux['N° de Estudantes'], 
                       df_aux['Pontuação A(%)'],df_aux['Pontuação B(%)'],df_aux['Pontuação D(%)']],
               fill_color='lavender',
               align='left'))
])

fig.update_layout(title_text="Demostrativo Quantidade de Estudantes por Estado considerando <b>Pontuação</b>",title_x=0.5) 
fig.show()

In [59]:
cross_tab_segmentacao = pd.crosstab(index=data_apa_duplicate['cod_estado'],
                             columns=data_apa_duplicate['level_segmentacao'])
cross_tab_segmentacao.reset_index(inplace=True)
cross_tab_segmentacao.columns = ['Estado','Pontuação A','Pontuação B','Pontuação C']

fig = go.Figure(data=[go.Table(header=dict(values=list(cross_tab_segmentacao.columns),
                fill_color='paleturquoise',
                align='left'),
               cells=dict(values=[cross_tab_segmentacao['Estado'],
                                  cross_tab_segmentacao['Pontuação A'],
                                  cross_tab_segmentacao['Pontuação B'],
                                  cross_tab_segmentacao['Pontuação C']],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(title_text="Demostrativo Quantidade de Alunos por Estado considerando <b>Segmento</b>",title_x=0.5) 
fig.show()

In [28]:
result = data_apa_duplicate.groupby(['state']).agg({'state':["count"]},split_out=4)
result.reset_index(inplace=True)
result.columns = ['Status da Digitalização','Quantidade']
fig = go.Figure(data=[go.Table(header=dict(values=list(result.columns),
                fill_color='paleturquoise',
                align='left'),
               cells=dict(values=[result['Status da Digitalização'],result['Quantidade']],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(title_text="Status Digitalizações da Base APA",title_x=0.5) 
fig.show()

In [11]:
##################Descritivo SAP##################
sql_aluno ='SELECT * FROM escolas.aluno'
sql_turma ='SELECT * FROM escolas.turma'
sql_escola ='SELECT * FROM escolas.escola'
#df = connection(sql_escola)
#data_aluno = pd.read_csv('/home/eltonss/Documents/MEC/data/aluno.csv');
data_aluno = connection(sql_aluno)
#data_turma = pd.read_csv('/home/eltonss/Documents/MEC/data/turma.csv'
data_turma  = connection (sql_turma)                    
#data_escola = pd.read_csv('/home/eltonss/Documents/MEC/data/escola.csv');
data_escola = connection(sql_escola)

In [12]:
data_aluno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076 entries, 0 to 14075
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_aluno       14076 non-null  int64  
 1   id_turma       14076 non-null  int64  
 2   cod_matricula  9 non-null      float64
 3   nome_turma     8 non-null      object 
 4   nome_aluno     14076 non-null  object 
 5   tp_sexo        8 non-null      object 
dtypes: float64(1), int64(2), object(3)
memory usage: 659.9+ KB


In [13]:
#Merge tabela aluno e turma pela chave estrangeira removendo colunas iguais nas tabelas
cols_to_use = data_turma.columns.difference(data_aluno.columns)
#print(cols_to_use)
data_aluno_turma = pd.merge(data_aluno, data_turma[cols_to_use], left_index=True, right_index=True, how='outer')
#data_aluno_turma = pd.merge(data_aluno, data_turma, on='id_turma')
data_aluno_turma.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14076 entries, 0 to 14075
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id_aluno       14076 non-null  int64  
 1   id_turma       14076 non-null  int64  
 2   cod_matricula  9 non-null      float64
 3   nome_turma     8 non-null      object 
 4   nome_aluno     14076 non-null  object 
 5   tp_sexo        8 non-null      object 
 6   ano_turma      382 non-null    object 
 7   data_criacao   382 non-null    object 
 8   etapa_ensino   360 non-null    object 
 9   id_escola      382 non-null    float64
 10  modo_ensino    360 non-null    object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.3+ MB


In [14]:
data_escola.rename(columns={'cod_escola': 'id_escola',}, inplace=True)


In [62]:
#Merge tabela escola com dataframe aluno_turma
cols_to_use = data_escola.columns.difference(data_aluno_turma.columns)
#print(cols_to_use)
data_aluno_turma_escola = pd.merge(data_aluno_turma, data_escola[cols_to_use], 
                                   left_index=True, right_index=True, how='outer')


In [16]:
dict_uf= {11:'RO',12:'AC',13:'AM',14:'RR',15:'PA',16:'AP',17:'TO',21:'MA',22:'PI',23:'CE',24:'RN',
              25:'PB',26:'PE',27:'AL',28:'SE',29:'BA',31:'MG',32:'ES',33:'RJ',35:'SP',41:'PR',42:'SC',43:'RS',50:'MS',
              51:'MT',52:'GO',53:'DF',}
data_aluno_turma_escola = data_aluno_turma_escola.replace({"cod_estado": dict_uf})

dict_municipio = {}
for code in set(data_aluno_turma_escola['municipio']):
    for codigo,nome in zip(municipios.id_municipio,municipios.nome):
        if(code==codigo):
            dict_municipio[code] = nome;
data_aluno_turma_escola=data_aluno_turma_escola.replace({'codigo_cidade': dict_municipio})



In [30]:
data_aluno_turma_escola
result = data_aluno_turma_escola.groupby(['cod_estado',]).agg({'id_aluno':["nunique"],"id_turma": ["nunique"],"municipio": ["nunique"],'id_escola': ["nunique"]},split_out=4)
result.reset_index(inplace=True)
result.columns = ['Estado','Quantidade de Alunos','Quantidade de Turmas','Quantidade de Cidades','Quantidade de Escolas']
fig = go.Figure(data=[go.Table(
    header=dict(values=list(result.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[result.Estado, result['Quantidade de Alunos'],result['Quantidade de Turmas'], result['Quantidade de Cidades'],result['Quantidade de Escolas']],
               fill_color='lavender',
               align='left'))
])
fig.update_layout(title_text="Descritivo por Estado da Base SAP",title_x=0.5) 

fig.show()